#Running SPUD with Profile and Progressions as Domains

In [1]:
#Import necessary libraries

import sys
import os

# Make it so you can get modules from the parent directory
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.insert(0, parent_dir)

import test_manifold_algorithms as tma
from MASH import MASH
from SPUD import SPUD
from temporal_progression_comparisons import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


This is not running as __main__


In [4]:
LABEL_VARIABLE = "DX_bl"
SAMPLE_SIZE = 20

profile_raw = pd.read_excel(r"..\..\CSV Files\Profile Variables 2024-07-31.xlsx", index_col="RID")
progressions_raw = pd.read_excel(r"..\..\CSV Files\Progression Variables 2024-08-02.xlsx", index_col = [0,1]) #multilevel index

#drop empty rows and fill in NaNs
profile_raw.dropna(how="all", inplace=True)
profile_raw = profile_raw.fillna(-4)
progressions_raw.dropna(how="all", inplace=True)
progressions_raw = progressions_raw.fillna(-4)

#remap and delete a few variables, might make part of the merge later to be more steamlined on when to map and delete stuff
profile_raw['DX_bl'] = profile_raw['DX_bl'].map({'CN': 0, 'SMC': 1, 'EMCI': 2, 'LMCI': 3, 'AD': 4, 'NaN': -4, -4: -4})
profile_raw['PTMARRY'] = profile_raw['PTMARRY'].map({'Married': 1, 'Widowed': 2, 'Divorced': 3, 'Never married': 4, 'Unknown': -4, 'NaN': -4, -4: -4})

profile = profile_raw.drop(columns = [LABEL_VARIABLE, "PTRACCAT"])
progressions = progressions_raw.drop(columns = [])

label_column = profile_raw[LABEL_VARIABLE]

In [18]:
#take the first SAMPLE_SIZE of people for domain_a
domain_a = np.array(profile)[:SAMPLE_SIZE].astype(float)

#get the unique rids in the sample and their corresponding visits for domain_b
sample_rids = list(profile.index.values)[:SAMPLE_SIZE]
progressions_sample = progressions.loc[progressions.index.get_level_values(0).isin(sample_rids)] #must be done this way because some of the sample_rids don't have any progressions
domain_b = progressions_sample #our custom comparison function takes a dataframe, so don't transform into array